# 23.4.2020
The idea is that during the last part, you got your multiple structure alignment **MStA** and we downloaded a Multiple sequ ence alignment **MSqA** of the structure we have selected. So we used the multiple sequence alignment of the structures we have selected. I think i got 13 after removing redundancy.

And then we did the structural alignment using the PDB identifyer using [PDBfold](https://www.ebi.ac.uk/msd-srv/ssm/) but you can also use a different method. But still it should yield more or less the same result. Having different constraint you can get slightly different number of structures.

The idea is that when we get the representation of MStA through the sequence alignment - we use this to generate the HMM using the basic comand in [HMMER](http://www.csb.yale.edu/userguides/seq/hmmer/docs/node19.html):

``` hmmbuild ```

You can use skyline or other types of servers to visualize the alignment and to check for conserved regions that are typical for this particular family such as finding the cysteins.

So in our case we can have a look at the

``` vi clean_bpti-kunitz-aln.fasta ```
So you can go there and check the locations of the cysteins;
![clean_bpti-kunitz-aln.fasta](./1.png)

So some cysteins are not there - maybe because of the cyristallization.
Try to see what happenes when you remove them. This is all stuff you can add into the discussion.

There is a conserved lysin (K) but they are at a slightly different position and you can check what importance it has for the funciton. And you should check all these positions and maybe include that into the conclusion. 

So we have generated our HMM, you should go and check how long it is:

In [1]:
!less bpti-kunitz.hmm # 58 in his case

When we generated the model we have generated the first part of the method. So we have amethod that is ready to detect the proteins containing the BPTI kunitz domains. Second step we need to generate a set for benchmarking the method. We have to test the ability of the method to really predict the proteins that contain such domain.
For doing this we went to Uniprot/Swissprot. To do this we did an advanced search selecting the a set of  positives; BPTI Kunitz and a set of Non-BPTI-Kunitz which are the negatives by just searching for proteins that do not contain the PFAM identifyer. Obviously for the negatives we got a lot more hits (almost the entire Uniport/SP DB) while for the positive ones we only got around 300+.
First we have to sort out **one issue**: the fact that the proteins that we have selected for generating the model are in our set of positives. This is why we need to attempt to **remove such** sequences from our set of positives. This was doen by runnin blast pgp. First we need to generate a dataset. In our case a set of all the pt that we have selected from the PDB. So in our case it should be  ```pdb_kunitz.fasta ```.

In [1]:
#!formatdb -i pdb_kunitz.fasta 
# for new of blast version its:
#!makeblastdb -in pdb_kunitz.fasta -dbtype prot

Then you need to run the positives against this set of sequences with blastpgp:
```clean_all_bpti_kunitz.fasta``` are all the 359 that we have selected.

``` 
grep '>' clean_all_bpti_kunitz.fasta |wc
359 2876
```

In [2]:
# old ver:
#blastpgp -i clean_all_bpti_kunitz.fasta -d pdb_kunitz.fasta -m8 -o clean_all_bpti_kunitz.bl8 -e 0.001

# new cmd version:
# blastp -query nr_all_bpti_kunitz.fasta  -db pdb_kunitz.fasta -out clean_all_bpti_kunitz.bl6 -outfmt 6 -evalue 10 

In [ ]:
You can also use the evalue of ```-e 0.001```to remove distantly related hits with high percentages.

Then you have to chech for the 3rd col because it is the seq identity.
![less clean_all_bpti_kunitz.bl8](./2.png)
Now you have to find the proteins that are 100% or close to 100%:
So we check if line 3 is greater than 95% pring the whole line

In [3]:
#awk '{if($3>=95) print $0}' clean_all_bpti_kunitz.bl8 | less

![seq that score higher than 95%](./3.png)

Last time he removed seq that had 99% - it doesnt matter you chose your cutoff...
[Notes from the previous day](https://docs.google.com/document/d/1A9mOpchenoduFIEaL1DcrcRnn9mlcszOLHIv9t6RCb8/edit?usp=sharing)

Now you need to pring only they first identifyer and sort -u and print that list:

In [ ]:
#awk '{if($3>=99) print $1}' clean_all_bpti_kunitz.bl8 |sort -u 
# you shoult get about 5 or 6.

![](./4.png)

From the list of ID's which we have to remove from our set of positives. So we take our fasta file of positives and remove the set of sequences that are redundant (they are the same as the ones we are using in our model). If they are few we can do it manually - but we already wrote a script in one of the previous lessons to select a particular group of sequences from a fasta file, which selects all **but** the ones provided in the list of ID's (extract_seq.py and neg_extract_seq.py). They can be modified to filter instead of selecting. So we only have to change it.

In [4]:
#the positive one was this one
# s == 0.0
# for j in range(20):
#     if pi[j]>0: s=s-pi[j]*np.log(pi[j])
# pm=pi.argmax()
# print(i+1, aa_list[pm], s, pi[pm], profile[i][20])

# if __name__ == '___main__':
#     alfile=sys.argv[1]
#     d_aln=get_aln(alnfile)
#     profile=get_profile(d_aln)
#     print_profile(profile)

In [5]:
# # the negative was 
# import sys 

# def get_ids(idfile):
#     ids=open(idfile).read().rstrip().split('\n')
#     return ids

# def print_seqs(ids, dbfile):
#     with open(dbfile, 'r') as fdb:
#         for line in fdb:
#             if line[0] == '>':
#                 #pid=line.split('|')[1]
#                 pid=line[1:].rstrip()
#             if pid not in ids:  # ids is the list of ids we want to select so we just add the 'not' to make it do the opposite 
#                 print line.rstrip()


With this basic script we run the comand below we should remove 5 of 395 so we should get back 354

In [ ]:
#! python ../neg_extract_seq.py redundant_set.txt clean_all_bpti_kunitz.fasta |grep '>' |wc

Now we have our 'cleaned' positives and all the negatives. 
We have to do training and testing on a different set.

Cross validation:
1. Training step
2. Testing step

Both steps are done on a set that is different from each other.
We want to **devide** our set into a group of n subsets. We take n-1 subsets and **train** our method thereafter we to **test** our method on a different set. By deviding our set into 2 we do a **2 fold corssvalidation** (usually at least 5 fold). 

To make sure the sequences are selected **randomly**, we did a procedure based on the sort cmd. /> Sort -r allows to shuffle your list of id's from the fasta file. Then you take out the first 50% and generate  first set of positives and then the 2nd set of positives. You can do this with our previous script;   the one that extracts the sq from a fasta file.

In [6]:
#!grep '>' nr_all_bpti_kunitz.fasta |wc #should return 354 sequences.

Then the idea is to recover the list of identifyers with the sed cmd:
You can save this sorted list wherever you want to, and then take out the first 177 and do the first set and then take the remaining ones to do the second set.

In [7]:
#!grep '>' nr_all_bpti_kunitz.fasta |sed 's>//' |less

#if you want to obtain a sorted list then motify to:
#!grep '>' nr_all_bpti_kunitz.fasta sed 's>//|sort -R

In [ ]:
2744

This is random sampling of the set of the ids
he called them ``` random1.txt ```

This list of ID's could be the input of that we can run on the ```extract_seq.py ```

In [14]:
# head -n 177 random1.txt
# tail +178 random1.txt
# python ../extract_seq.py <(head -n 177  random1.txt) nr_allbpti_kunitz.fasta>|grep ">" |wc 
# 177 177 1420
#this one is for the set number 2:
# python ../extract_seq.py <(tail -n +178 random1.txt) nr_allbpti_kunitz.fasta| grep ">" |wc 


So now we have generated the positive_set1.txt, positive_set2.txt (both composed of 177) and then for the negatives:

If we have our list of positves, that are now not this non redundant but **all**

This would be your initial list of identifiers 
 

In [ ]:
!grep '>' clean_all_bpti_kunitz.bl8 |sed 's/>//'

And you can run this list of ID 
So if you want to select only the negatives from the uniprot_sprot.fasta 

If you want to clean this one (that means select only the ids that match) 
![](./5.png)


or slightly modify the neg_extract_seq.py file in such a way that it matches the identifier with the second position. Remember that in our script we have a way to select the pid       
and change line 14 below so this way we clean the ID to make the output more readable.

neg_extract_seq.py

In [ ]:
# # the negative was  neg_extract_seq.py

# import sys 

# def get_ids(idfile):
#     ids=open(idfile).read().rstrip().split('\n')
#     return ids

# def print_seqs(ids, dbfile):
#     with open(dbfile, 'r') as fdb:
#         for line in fdb:
#             if line[0] == '>':
#                 #pid=line.split('|')[1]
#                 pid=line[1:].rstrip() #this way we clean the ID to make the output more readable
#             if pid not in ids:  # ids is the list of ids we want to select so we just add the 'not' to make it do the opposite 
#                 print line.rstrip()


In [15]:
#!awk '{if substr($0,1,1)=='>') {split($0,a,'|');print '>'a[2]}}'  ../uniprot_sprot.fasta |less

That way you get a clean file in such a way that you can get a full list.
For the second part:

In [ ]:
#!awk '{if substr($0,1,1)=='>') {split($0,a,'|');print '>'a[2]}}' else print $0 ../uniprot_sprot.fasta |less

That way you get a clean SP file and you can run the negative extract on  the list of the 359 in such a way that you get the negatives and then you can split them in 2.
He actually generated a set of human negatives -> from the set of **only human non BPTI** he called it negative_set1.txt and it contained 10 000 and the other part a little more than 10 000

In [16]:
# !grep '>' negative_set!.txt |wc
# !grep '>' negative_set2.txt |wc

You could also remove seqs that are contain **fewer than 20 residues** to improve your set. In Profs case there are 10000.

# Run The Algorithm for Searching the Matches
There are some options that could be useful to run the cmd in an appropritate way: 
lets have a look in the help pages.

In [8]:
#!hhmsearch -h |less 

In [18]:
# tblout #saving the output in tabular format.
# --noali if you just want to know the scores.
# --max Turns all heuristic filters off (slower but more power)
# --Z is important because HHM search compares a HM-model against a seq DB

![](./6.png)

Remember that the calculation of the **e-value depends** on the **size** of the data set which was used for the search. So if the data set is large it will return a different e-value than when the dataset is small.

In [9]:
#  !hmmsearch --noali --max --tblout negative_set1.out bpti-kunitz.hmm negative_set1.txt
# so now we run on the negative_set1 forst.

![](7.png)

And we see that the first sequence got an e-value of 0.21 (full seq) and 0.54 for the best domain.

![](8.png)

If we now take this sequence copy it and (```P54762```) save it as seq.fasta

We take the seq for which we got the first hit and run the seqence.
43 what is the **difference** between the evaues of the diff or 4 orders of magnitiude.
Because the 2 sets have different sizes. This happenes without any modification of particular options.

If there are many more negatives (around 200 000 in our case), the **evalue changes**.

**Q:** So basically the new -Z option makes the E-value computation independent from the set size?
The idea is that we have to compare parameters that are the same for everything. For example if we run it with the option ``` -Z = 1 ``` for the search:
- All calculations are done as if we only provided one sequence as an input.

**Q:** So we are **setting the search space**? Yes, since we are not interested in a real value. So we create a measure that is comparable. So you have to find out what is common for all.

In [20]:
# !hhmsearch -Z 1 --noali --max --tblout negative_set1.out bpti-kunitz.hmm negative_set1
# !hhmsearch -Z 1 --noali --max --tblout negative_set2.out bpti-kunitz.hmm negative_set2
# !hhmsearch -Z 1 --noali --max --tblout positive_set1.out bpti-kunitz.hmm positive_set1
# !hhmsearch -Z 1 --noali --max --tblout positive_set2.out bpti-kunitz.hmm positive_set2

Now we need to parse this file negative_set1.out 
to get the ID and the e-value:
So if we select the e-value of the 'best 1 domain' $8:

In [21]:
# ! grep -v '^#' negative_set1.out |awk '{print $1,$8}' |less

In preparation of the calculation of the calculation of the performance (53)

In [10]:
# !grep -v '^#' negative_set1.hits |awk '{print $1,$8, 0}' |less >negative_set1.out
# !grep -v '^#' negative_set2.hits |awk '{print $1,$8, 0}' |less >negative_set2.out
# !grep -v '^#' positive_set2.hits |awk '{print $1,$8, 1}' |less >positive_set2.out
# !grep -v '^#' positive_set1.hits |awk '{print $1,$8, 1}' |less >positive_set1.out

Now we check if the number is reasonable:
    
![](9.png)


Now we **reintroduce** the hits that are missing. You can do this in many ways, he did it with the com cmd or the sed cmd

In [12]:
# this is just for expalnation; he didnt want to write everything
# diff.py:
# import sys

# f1=open(sys.argv[1])
# f2=open(sys.argv[2])

# s1 = set([i.rstrip() for i in f1])
# s2 = set([i.rstrip() for i in f2])

# d = s1-s2

# for i in list(d):
#     print(i)

With basic scripts like that you can check for the differences or the union.
The idea is that when you have your list of ID in the negative set:


In [13]:
#!grep '>' negative_set1.txt |sed 's/>//' | less
#!python diff.py <(grep ">" negative_set1.txt |sed 's/>//') <(awk '{print $1}' negative_set1.out)|less
#!python diff.py <(grep ">" negative_set1.txt |sed 's/>//') <(awk '{print $1}' negative_set1.out) |awk '{print $1, 10, 0}' >ok_negative_set1.out

In [14]:
# in line 3 above the  '10' stands for the evalue |awk '{print $1, 10, 0}' 

If you sum up these 2 numbers you should get the full set.

[](10.png)

Now we see the positive and negative are ok - and now we have the performance.py


In [ ]:
#!python ../blast/performance.py <(cat positive_set1.out ok_negative_set1.out)

[](./11.png)


With respect to what we did yesterday the range of which we get the best Mathew Coefficient Correleation - it goes now from 10^-5 to 10^10. Because now we are **rescaling the evalue** threshold gets a little bit lower. 

When you do you own you will use a **larger dataset** so the previously mentioned range should be a little bit smaller. You have to **determine threashold** from this and apply it to the set 2 and apply the best threashold to the set1.
 
If you whish you can use the 2 results (= the average value between them, which is your performacne in a testing mode.

This way you can evaluate weather the performance in **testing mode** is differnet from the performance in training mode. The performance in testing mode should be slightly lower than the one from training mode.

I suggest you write your own library for the calculation of the **performance**

Obviously if you want to test more measures you can check out [scikit learn](https://scikit-learn.org/stable/)

23.4th and 25.4th we have to work at the project so we have time to ask questions.

In the report add a table that shows the value of performance.
So you need a tbl that shows that you have done optimization. Maybe you dont need to write all the table but mention a *group of threasholds* for which the performance is ok.
In your case you may also get some **false positives** (probably 1 or 2) you need to check and report why you got them.When you are in a situation where there are only few then maybe you find particular domains that are closely related to Kunitz.

- Do **not** add comands
- You can add the sequencelogo 
- Table how the data set is made up (positives, negatives...)
- Discussion: regions that are important: see if they are conserved or not conserved
- Results: all tables all the final results that you got
- As a referencing tool use [Zotero](https://www.zotero.org/)

**Q** We obtain a list of results for negative_set1 + positive_set1 and a list of results for negative_set2 + positive_set2: we have to check if the reults are similar?

You will get a list of results:
- list or res set 1 
- list of res set 2

The idea is to pick the best one of set1 and see what is the performance among the result for this threashold on set 2. In this way you will get an idea of the performance in the testing best threashold.
And then you it the other way around (what does he mean?? video 1:18:00). So you need to make sure that  the same threashold that was best for s1, is also the best for s2.



# How to submit? 
- make a directory and add:  
- the paper
- folder suplementary material
- zip it and send it.
- or share it via sharepoint/ gdrive/ github what ever floats your boat :) 
- the **only restraint**: write the **name of the folder properly**.
- google docs are compatible with zotero :)


### Fun Trick
If you want to do a 'reverse-i-search' so a search in the history of your cmds:

press ```ctr + r``` in the terminal then you can type a comand that you only vaguely remember and it will match the commands you've used from the history. So it is enough if you only remember one word in it  :-)